## Import required packages

In [15]:
from nltk.corpus import reuters
import nltk
nltk.download('wordnet')
nltk.download('reuters')
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from yellowbrick.text import TSNEVisualizer
import warnings
warnings.filterwarnings("ignore")
import torch
from torch import nn, optim
import pickle

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [16]:
train_documents, train_categories = zip(*[(reuters.raw(i), reuters.categories(i)) for i in reuters.fileids() if i.startswith('training/')])
test_documents, test_categories = zip(*[(reuters.raw(i), reuters.categories(i)) for i in reuters.fileids() if i.startswith('test/')])
print("Number of training documents:", len(train_documents))
print("Number of testing documents:", len(test_documents))
print(train_documents[0])
print(train_categories[0])

Number of training documents: 7769
Number of testing documents: 3019
BAHIA COCOA REVIEW
  Showers continued throughout the week in
  the Bahia cocoa zone, alleviating the drought since early
  January and improving prospects for the coming temporao,
  although normal humidity levels have not been restored,
  Comissaria Smith said in its weekly review.
      The dry period means the temporao will be late this year.
      Arrivals for the week ended February 22 were 155,221 bags
  of 60 kilos making a cumulative total for the season of 5.93
  mln against 5.81 at the same stage last year. Again it seems
  that cocoa delivered earlier on consignment was included in the
  arrivals figures.
      Comissaria Smith said there is still some doubt as to how
  much old crop cocoa is still available as harvesting has
  practically come to an end. With total Bahia crop estimates
  around 6.4 mln bags and sales standing at almost 6.2 mln there
  are a few hundred thousand bags still in the hands of 

In [17]:
print(train_documents[0])

BAHIA COCOA REVIEW
  Showers continued throughout the week in
  the Bahia cocoa zone, alleviating the drought since early
  January and improving prospects for the coming temporao,
  although normal humidity levels have not been restored,
  Comissaria Smith said in its weekly review.
      The dry period means the temporao will be late this year.
      Arrivals for the week ended February 22 were 155,221 bags
  of 60 kilos making a cumulative total for the season of 5.93
  mln against 5.81 at the same stage last year. Again it seems
  that cocoa delivered earlier on consignment was included in the
  arrivals figures.
      Comissaria Smith said there is still some doubt as to how
  much old crop cocoa is still available as harvesting has
  practically come to an end. With total Bahia crop estimates
  around 6.4 mln bags and sales standing at almost 6.2 mln there
  are a few hundred thousand bags still in the hands of farmers,
  middlemen, exporters and processors.
      There are doubt

In [18]:
s = set()
for i in train_categories:
        for j in i:
            s.add(j)
print(len(s))

90


# Convert the categorical labels to Multi Label Encodings

In [19]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(train_categories)
test_labels = mlb.transform(test_categories)
list_trainlabels = []
list_testlabels = []
for i in train_labels:
    list_trainlabels.append(list(i))
for i in test_labels:
    list_testlabels.append(list(i))
train_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [5]:
with open('multilable.pkl', 'wb') as file:
    pickle.dump(mlb, file)

## Creating Dataframe for applying transformations

In [20]:
trainData = {"content": train_documents}
testData = {"content": test_documents}
trainDf = pd.DataFrame(trainData, columns=["content"])
testDf = pd.DataFrame(testData, columns=["content"])

# Pre-process the input data

In [21]:
wordnet_lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stopwords = set(w.rstrip() for w in open("./reuters/stopwords"))

def tokenize_lemma_stopwords(text):
    text = text.replace("\n", " ")
    tokens = nltk.tokenize.word_tokenize(text.lower()) # split string into words (tokens)
    tokens = [t for t in tokens if t.isalpha()] # keep strings with only alphabets
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [stemmer.stem(t) for t in tokens]
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    cleanedText = " ".join(tokens)
    return cleanedText

def dataCleaning(df):
    data = df.copy()
    data["content"] = data["content"].apply(tokenize_lemma_stopwords)
    return data

In [22]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [23]:
cleanedTrainData = dataCleaning(trainDf)
cleanedTestData = dataCleaning(testDf)

In [24]:
cleanedTrainData['content'][0]

'bahia cocoa review shower continu week bahia cocoa zone allevi drought sinc earli januari improv prospect temporao normal humid level restor comissaria smith weekli review dri period temporao late thi year arriv week end februari bag kilo make cumul total season mln stage year cocoa deliv earlier consign includ arriv figur comissaria smith doubt crop cocoa avail harvest practic end total bahia crop estim mln bag sale stand mln hundr thousand bag hand farmer middleman export processor doubt thi cocoa fit export shipper experienc dificulti obtain certif view lower qualiti recent week farmer sold good part cocoa held consign comissaria smith spot bean price rose cruzado arroba kilo bean shipper reluct offer nearbi shipment onli limit sale book march shipment dlr tonn port crop sale light open port dlr dlr york juli dlr tonn fob routin sale butter made sold dlr butter time york dlr dlr time york sept dlr time york dec comissaria smith destin covert currenc area uruguay open port cake sale

# Create TF-IDF Vectorizer

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

vectorizer = TfidfVectorizer()
vectorised_train_documents = vectorizer.fit_transform(cleanedTrainData["content"])
vectorised_test_documents = vectorizer.transform(cleanedTestData["content"])

In [26]:
vectorised_train_documents

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 324791 stored elements and shape (7769, 16909)>

In [15]:
import pickle

with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

# Train and Evaluate Classifiers

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np
from tqdm import tqdm
import scipy

def prepare_text_data(texts_train,texts_test, labels_train, labels_test):
    tfidf = TfidfVectorizer(max_features=10000)  
    X_train = tfidf.fit_transform(texts_train).toarray()
    X_test = tfidf.transform(texts_test).toarray()
    
    mlb = MultiLabelBinarizer()
    y_train = mlb.fit_transform(labels_train)
    y_test = mlb.transform(labels_test)
    
    return X_train, X_test, y_train, y_test, tfidf, mlb

class ANN(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_in, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, n_out),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.net(x)

class MultiLabelTrainer:
    def __init__(self, model, device, learning_rate=0.001):
        self.model = model
        self.device = device
        self.criterion = nn.BCELoss()
        self.optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='min', patience=3, factor=0.1, verbose=True
        )

    def train_epoch(self, train_loader):
        self.model.train()
        total_loss = 0
        
        for batch_x, batch_y in tqdm(train_loader, desc="Training"):
            batch_x = batch_x.to(self.device)
            batch_y = batch_y.to(self.device)
            
            self.optimizer.zero_grad()
            outputs = self.model(batch_x)
            loss = self.criterion(outputs, batch_y)
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
        
        return total_loss / len(train_loader)

    def evaluate(self, data_loader, threshold=0.5):
        self.model.eval()
        all_predictions = []
        all_targets = []
        total_loss = 0
        
        with torch.no_grad():
            for batch_x, batch_y in tqdm(data_loader, desc="Evaluating"):
                batch_x = batch_x.to(self.device)
                batch_y = batch_y.to(self.device)
                
                outputs = self.model(batch_x)
                loss = self.criterion(outputs, batch_y)
                total_loss += loss.item()
                
                predictions = (outputs >= threshold).float()
                all_predictions.extend(predictions.cpu().numpy())
                all_targets.extend(batch_y.cpu().numpy())

        all_predictions = np.array(all_predictions)
        all_targets = np.array(all_targets)
        
        f1 = f1_score(all_targets, all_predictions, average='macro')
        precision = precision_score(all_targets, all_predictions, average='macro')
        recall = recall_score(all_targets, all_predictions, average='macro')
        accuracy = accuracy_score(all_targets, all_predictions)
        
        return {
            'loss': total_loss / len(data_loader),
            'accuracy' : accuracy,
            'f1': f1,
            'precision': precision,
            'recall': recall
        }

def prepare_data(X_train, y_train, X_test, y_test, batch_size=32):
    if scipy.sparse.issparse(X_train):
        X_train = X_train.toarray()
    if scipy.sparse.issparse(X_test):
        X_test = X_test.toarray()

    train_dataset = TensorDataset(
        torch.FloatTensor(X_train),
        torch.FloatTensor(y_train)
    )
    test_dataset = TensorDataset(
        torch.FloatTensor(X_test),
        torch.FloatTensor(y_test)
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader

def train_model(X_train, y_train, X_test, y_test, epochs=50, batch_size=32):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    n_features = X_train.shape[1]  
    n_classes = y_train.shape[1]   

    train_loader, test_loader = prepare_data(
        X_train, y_train, X_test, y_test, batch_size
    )

    model = ANN(n_features, n_classes).to(device)
    trainer = MultiLabelTrainer(model, device)
    
    best_train_loss = float('inf')
    best_model = None
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        train_loss = trainer.train_epoch(train_loader)
        train_metrics = trainer.evaluate(train_loader)
        
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Accuracy: {train_metrics['accuracy']:.4f}")
        print(f"Train F1: {train_metrics['f1']:.4f}")
        print(f"Train Precision: {train_metrics['precision']:.4f}")
        print(f"Train Recall: {train_metrics['recall']:.4f}")
        
        # Learning rate scheduling
        trainer.scheduler.step(train_loss)
        
        # Save best model based on training loss
        if train_loss < best_train_loss:
            best_train_loss = train_loss
            best_model = model.state_dict().copy()
    
    # Load best model and evaluate on test set
    model.load_state_dict(best_model)
    test_metrics = trainer.evaluate(test_loader)
    
    print("\nTest Set Results:")
    print(f"Test Accuracy: {test_metrics['accuracy']:.4f}")
    print(f"Test F1: {test_metrics['f1']:.4f}")
    print(f"Test Precision: {test_metrics['precision']:.4f}")
    print(f"Test Recall: {test_metrics['recall']:.4f}")
    
    return model, test_metrics

# Usage:
"""
# X_train: TF-IDF features for training
# X_test: TF-IDF features for testing
# y_train: Multi-label encoded labels for training
# y_test: Multi-label encoded labels for testing

# Train the model
model, metrics = train_model(
    X_train, y_train,
    X_test, y_test,
    epochs=50,
    batch_size=32
)

# Save the model
torch.save({
    'model_state_dict': model.state_dict(),
}, 'model_checkpoint.pth')
"""

"\n# X_train: TF-IDF features for training\n# X_test: TF-IDF features for testing\n# y_train: Multi-label encoded labels for training\n# y_test: Multi-label encoded labels for testing\n\n# Train the model\nmodel, metrics = train_model(\n    X_train, y_train,\n    X_test, y_test,\n    epochs=50,\n    batch_size=32\n)\n\n# Save the model\ntorch.save({\n    'model_state_dict': model.state_dict(),\n}, 'model_checkpoint.pth')\n"

In [17]:
model, metrics = train_model(
    vectorised_train_documents, train_labels,
    vectorised_test_documents, test_labels,
    epochs=50,
    batch_size=32
)
torch.save({
    'model_state_dict': model.state_dict(),
}, 'model_checkpoint.pth')

Using device: cuda

Epoch 1/50


Evaluating: 100%|██████████| 243/243 [00:00<00:00, 842.06it/s]


Train Loss: 0.0752
Accuracy: 0.7442
Train F1: 0.1346
Train Precision: 0.1539
Train Recall: 0.1240

Epoch 2/50


Evaluating: 100%|██████████| 243/243 [00:00<00:00, 1011.02it/s]


Train Loss: 0.0195
Accuracy: 0.8213
Train F1: 0.2478
Train Precision: 0.2894
Train Recall: 0.2333

Epoch 3/50


...

Train Loss: 0.0016
Accuracy: 0.9847
Train F1: 0.9891
Train Precision: 0.9849
Train Recall: 0.9948

Epoch 50/50


Evaluating: 100%|██████████| 243/243 [00:00<00:00, 942.62it/s]


Train Loss: 0.0014
Accuracy: 0.9848
Train F1: 0.9883
Train Precision: 0.9825
Train Recall: 0.9960


Evaluating: 100%|██████████| 95/95 [00:00<00:00, 818.51it/s]


Test Set Results:
Test Accuracy: 0.8155
Test F1: 0.5590
Test Precision: 0.6711
Test Recall: 0.5189


In [7]:
import torch
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

# Load the trained model's state dictionary
model_path = './model_checkpoint.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ANN(n_in=16909, n_out=90).to(device)

# Load and fix the state dict
checkpoint = torch.load(model_path, map_location=device)
if 'model_state_dict' in checkpoint:
    model.load_state_dict(checkpoint['model_state_dict'])
else:
    model.load_state_dict(checkpoint)

model.eval()

# Load the TfidfVectorizer and MultiLabelBinarizer from training
tfidf_path = './tfidf_vectorizer.pkl'
mlb_path = './multilable.pkl'
with open(tfidf_path, 'rb') as f:
    tfidf = pickle.load(f)
with open(mlb_path, 'rb') as f:
    mlb = pickle.load(f)

# Test text
test_text = [
    "Connecting to Redmi Buds 6 Active is also very simple, you just need to turn on Bluetooth on your phone and open the headphone case. Then you hold down the headphone power button for a moment so that the light flashes, then you can scan Bluetooth on your phone. This product uses Bluetooth 5.3 connection standard, so the product has high stability when used."
]

# Preprocess the text
test_text_preprocessed = [text.lower() for text in test_text]

# Vectorize the text
X_test = tfidf.transform(test_text_preprocessed)

# Convert to tensor and move to appropriate device
X_test_tensor = torch.FloatTensor(X_test.toarray()).to(device)

# Make prediction
with torch.no_grad():
    outputs = model(X_test_tensor)

# Get probabilities
probabilities = outputs  # outputs already have sigmoid in the model

# Get top 5 probabilities and their indices
top5_values, top5_indices = torch.topk(probabilities, k=5)

# Move tensors to CPU for numpy conversion
top5_values = top5_values.cpu()
top5_indices = top5_indices.cpu()

# Print top 5 labels and their probabilities
print("\nTop 5 Predicted Labels and Probabilities:")
for i in range(len(test_text)):
    for label_idx, prob in zip(top5_indices[i], top5_values[i]):
        label = mlb.classes_[label_idx]
        print(f"  {label}: {prob:.4f}")


Top 5 Predicted Labels and Probabilities:
  earn: 0.3866
  acq: 0.2811
  livestock: 0.0250
  trade: 0.0181
  ipi: 0.0100


In [27]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np
from tqdm import tqdm

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, n_out):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.5 if num_layers > 1 else 0
        )
        
        self.net = nn.Sequential(
            nn.Linear(hidden_size * 2, 512),  
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, n_out),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = x.unsqueeze(1)  # Add sequence dimension
        lstm_out, (hidden, _) = self.lstm(x)
        hidden_cat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        output = self.net(hidden_cat)
        return output

class MultiLabelTrainer:
    def __init__(self, model, device, learning_rate=0.001):
        self.model = model
        self.device = device
        self.criterion = nn.BCELoss()
        self.optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='min', patience=3, factor=0.1, verbose=True
        )

    def train_epoch(self, train_loader):
        self.model.train()
        total_loss = 0
        
        for batch_x, batch_y in tqdm(train_loader, desc="Training"):
            batch_x = batch_x.to(self.device)
            batch_y = batch_y.to(self.device)
            
            self.optimizer.zero_grad()
            outputs = self.model(batch_x)
            loss = self.criterion(outputs, batch_y)
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
        
        return total_loss / len(train_loader)

    def evaluate(self, data_loader, threshold=0.5):
        self.model.eval()
        all_predictions = []
        all_targets = []
        total_loss = 0
        
        with torch.no_grad():
            for batch_x, batch_y in tqdm(data_loader, desc="Evaluating"):
                batch_x = batch_x.to(self.device)
                batch_y = batch_y.to(self.device)
                
                outputs = self.model(batch_x)
                loss = self.criterion(outputs, batch_y)
                total_loss += loss.item()
                
                predictions = (outputs >= threshold).float()
                all_predictions.extend(predictions.cpu().numpy())
                all_targets.extend(batch_y.cpu().numpy())

        all_predictions = np.array(all_predictions)
        all_targets = np.array(all_targets)
        
        return {
            'loss': total_loss / len(data_loader),
            'accuracy': accuracy_score(all_targets, all_predictions),
            'f1': f1_score(all_targets, all_predictions, average='macro'),
            'precision': precision_score(all_targets, all_predictions, average='macro'),
            'recall': recall_score(all_targets, all_predictions, average='macro')
        }

def prepare_data(X_train, y_train, X_test, y_test, batch_size=32):
    if scipy.sparse.issparse(X_train):
        X_train = X_train.toarray()
    if scipy.sparse.issparse(X_test):
        X_test = X_test.toarray()

    train_dataset = TensorDataset(
        torch.FloatTensor(X_train),
        torch.FloatTensor(y_train)
    )
    test_dataset = TensorDataset(
        torch.FloatTensor(X_test),
        torch.FloatTensor(y_test)
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader

def train_model(X_train, y_train, X_test, y_test, epochs=50, batch_size=32):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    input_size = X_train.shape[1]  # Number of features
    n_classes = y_train.shape[1]   # Number of classes

    train_loader, test_loader = prepare_data(
        X_train, y_train, X_test, y_test, batch_size
    )

    model = LSTMClassifier(
        input_size=input_size,
        hidden_size=256,
        num_layers=2,
        n_out=n_classes
    ).to(device)
    
    trainer = MultiLabelTrainer(model, device)
    best_train_loss = float('inf')
    best_model = None

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")

        train_loss = trainer.train_epoch(train_loader)

        train_metrics = trainer.evaluate(train_loader)
        
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Accuracy: {train_metrics['accuracy']:.4f}")
        print(f"Train F1: {train_metrics['f1']:.4f}")
        print(f"Train Precision: {train_metrics['precision']:.4f}")
        print(f"Train Recall: {train_metrics['recall']:.4f}")

        trainer.scheduler.step(train_loss)

        if train_loss < best_train_loss:
            best_train_loss = train_loss
            best_model = model.state_dict().copy()

    model.load_state_dict(best_model)
    test_metrics = trainer.evaluate(test_loader)
    
    print("\nTest Set Results:")
    print(f"Test Accuracy: {test_metrics['accuracy']:.4f}")
    print(f"Test F1: {test_metrics['f1']:.4f}")
    print(f"Test Precision: {test_metrics['precision']:.4f}")
    print(f"Test Recall: {test_metrics['recall']:.4f}")
    
    return model, test_metrics

# Usage :
"""
# Your data should already be prepared as:
# X_train: TF-IDF features for training
# X_test: TF-IDF features for testing
# y_train: Multi-label encoded labels for training
# y_test: Multi-label encoded labels for testing

# Train the model
model, metrics = train_model(
    X_train, y_train,
    X_test, y_test,
    epochs=50,
    batch_size=32
)

# Save the model
torch.save({
    'model_state_dict': model.state_dict(),
}, 'lstm_reuters_model.pth')
"""

"\n# Your data should already be prepared as:\n# X_train: TF-IDF features for training\n# X_test: TF-IDF features for testing\n# y_train: Multi-label encoded labels for training\n# y_test: Multi-label encoded labels for testing\n\n# Train the model\nmodel, metrics = train_model(\n    X_train, y_train,\n    X_test, y_test,\n    epochs=50,\n    batch_size=32\n)\n\n# Save the model\ntorch.save({\n    'model_state_dict': model.state_dict(),\n}, 'lstm_reuters_model.pth')\n"

In [28]:
model, metrics = train_model(
    vectorised_train_documents, train_labels,
    vectorised_test_documents, test_labels,
    epochs=50,
    batch_size=32
)
torch.save({
    'model_state_dict': model.state_dict(),
}, 'lstm_reuters_model.pth')

Using device: cuda

Epoch 1/50


Evaluating: 100%|██████████| 243/243 [00:00<00:00, 530.05it/s]


Train Loss: 0.0806
Accuracy: 0.6728
Train F1: 0.0777
Train Precision: 0.0838
Train Recall: 0.0790

Epoch 2/50


Evaluating: 100%|██████████| 243/243 [00:00<00:00, 549.26it/s]


Train Loss: 0.0231
Accuracy: 0.7544
Train F1: 0.1548
Train Precision: 0.1850
Train Recall: 0.1521

Epoch 3/50


...

Train Loss: 0.0021
Accuracy: 0.9790
Train F1: 0.9713
Train Precision: 0.9701
Train Recall: 0.9786

Epoch 50/50


Evaluating: 100%|██████████| 243/243 [00:00<00:00, 558.93it/s]


Train Loss: 0.0021
Accuracy: 0.9799
Train F1: 0.9566
Train Precision: 0.9598
Train Recall: 0.9567


Evaluating: 100%|██████████| 95/95 [00:00<00:00, 470.97it/s]



Test Set Results:
Test Accuracy: 0.8095
Test F1: 0.5236
Test Precision: 0.6581
Test Recall: 0.4682


### WanDB

In [34]:
import wandb

wandb.login(
    key = "0f17446d98968f65557608f721190340ed0958e5",
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\DELL\_netrc


True

In [45]:

wandb.init(project="multi-label-text-classification", name="ANN1-Model")
wandb.config = {
    "learning_rate": 0.001,
    "batch_size": 32,
    "epochs": 50,
    "n_features": vectorised_train_documents.shape[1],
    "n_classes": 90
}

In [46]:
def train_model(X_train, y_train, X_test, y_test, epochs=50, batch_size=32):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    n_features = X_train.shape[1]  
    n_classes = y_train.shape[1]   

    train_loader, test_loader = prepare_data(
        X_train, y_train, X_test, y_test, batch_size
    )

    model = ANN(n_features, n_classes).to(device)
    trainer = MultiLabelTrainer(model, device)
    
    best_train_loss = float('inf')
    best_model = None

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        
        train_loss = trainer.train_epoch(train_loader)
        train_metrics = trainer.evaluate(train_loader)
        
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Accuracy: {train_metrics['accuracy']:.4f}")
        print(f"Train F1: {train_metrics['f1']:.4f}")
        print(f"Train Precision: {train_metrics['precision']:.4f}")
        print(f"Train Recall: {train_metrics['recall']:.4f}")

        wandb.log({
            "Train Loss": train_loss,
            "Train Accuracy": train_metrics['accuracy'],
            "Train F1": train_metrics['f1'],
            "Train Precision": train_metrics['precision'],
            "Train Recall": train_metrics['recall']
        })
        
        trainer.scheduler.step(train_loss)
        
        if train_loss < best_train_loss:
            best_train_loss = train_loss
            best_model = model.state_dict().copy()
            torch.save(best_model, 'model_checkpoint1.pth')
    
        model.load_state_dict(best_model)
        test_metrics = trainer.evaluate(test_loader)

        wandb.log({
            "Test Accuracy": test_metrics['accuracy'],
            "Test F1": test_metrics['f1'],
            "Test Precision": test_metrics['precision'],
            "Test Recall": test_metrics['recall']
        })

    artifact = wandb.Artifact('best_model', type='model')
    artifact.add_file('model_checkpoint1.pth')
    wandb.log_artifact(artifact)
    
    wandb.finish()
    
    return model, test_metrics

In [47]:
train_model(
    vectorised_train_documents, train_labels,
    vectorised_test_documents, test_labels,
    epochs=50,
    batch_size=32
)

Using device: cuda

Epoch 1/50


Evaluating: 100%|██████████| 243/243 [00:00<00:00, 792.32it/s]


Train Loss: 0.0749
Accuracy: 0.7404
Train F1: 0.1327
Train Precision: 0.1769
Train Recall: 0.1212


Evaluating: 100%|██████████| 95/95 [00:00<00:00, 614.20it/s]



Epoch 2/50


...

Evaluating: 100%|██████████| 243/243 [00:00<00:00, 334.93it/s]


Train Loss: 0.0015
Accuracy: 0.9843
Train F1: 0.9870
Train Precision: 0.9805
Train Recall: 0.9951


Evaluating: 100%|██████████| 95/95 [00:00<00:00, 297.62it/s]


Test Accuracy,▁▃▅▅▆▇▇▇▇▇▇▇▇██████████▇█▇▇▇▇███████████
Test F1,▁▂▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇█▇████████▇████████████
Test Precision,▁▂▃▄▅▆▆▆▆▇▇▇▇▇▇▇▇███████▇█▇█████████████
Test Recall,▁▂▃▄▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇██▇▇▇█▇▇█████████████
Train Accuracy,▁▄▅▅▆▇▇▇▇▇██████████████████████████████
Train F1,▁▂▃▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇█████████████████████
Train Loss,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Precision,▁▂▃▄▄▅▅▆▆▆▇▆▇▇▇▇▇███████████████████████
Train Recall,▁▂▃▃▄▅▅▅▅▆▆▆▇▇▇▇▇▇██████████████████████
Test Accuracy,0.81981
Test F1,0.54996


(ANN(
   (net): Sequential(
     (0): Linear(in_features=16909, out_features=512, bias=True)
     (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (2): ReLU(inplace=True)
     (3): Dropout(p=0.5, inplace=False)
     (4): Linear(in_features=512, out_features=256, bias=True)
     (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (6): ReLU(inplace=True)
     (7): Dropout(p=0.5, inplace=False)
     (8): Linear(in_features=256, out_features=90, bias=True)
     (9): Sigmoid()
   )
 ),
 {'loss': 0.01526783846952879,
  'accuracy': 0.819807883405101,
  'f1': 0.5499635672204606,
  'precision': 0.6660652228462178,
  'recall': 0.5061886963713994})